# Solana Manager Tools

Este notebook contiene herramientas completas para gestionar wallets, transferencias y utilidades de Solana.

### Importaciones y Configuración

In [1]:
from solana_manager import SolanaWalletManager, SolanaAccountInfo, SolanaTransfer, SolanaUtils
import os

In [ ]:
# Configuración de red
NETWORK = "mainnet-beta"
RPC_URL = "https://api.mainnet-beta.solana.com"

# Inicializar manager principal
wallet_manager = SolanaWalletManager(network=NETWORK, rpc_url=RPC_URL)

# Inicializar herramientas
account_info = SolanaAccountInfo(wallet_manager)
transfer_manager = SolanaTransfer(wallet_manager)
utils = SolanaUtils(wallet_manager)

print(f"✅ Solana Manager inicializado en {NETWORK}")


## 1. Gestión de Wallets

Herramientas para crear, cargar y gestionar wallets de Solana.

### 1.1 Crear Nueva Wallet

In [ ]:
# Crear nueva wallet y guardarla en archivo
wallet_filename = wallet_manager.create_wallet_file()

if wallet_filename:
    print(f"\n📁 Wallet guardada en: {wallet_filename}")
    print(f"📍 Dirección: {wallet_manager.get_address()}")
    print("⚠️  IMPORTANTE: Guarda este archivo de forma segura")


### 1.2 Cargar Wallet Existente

In [ ]:
# Cargar wallet desde archivo (cambia por tu archivo)
WALLET_FILE = "wallets/mi_wallet_20250611_192531.json"

if os.path.exists(WALLET_FILE):
    success = wallet_manager.load_wallet(WALLET_FILE)
    if success:
        print(f"✅ Wallet cargada exitosamente")
        print(f"📍 Dirección: {wallet_manager.get_address()}")
    else:
        print("❌ Error cargando wallet")
else:
    print(f"❌ Archivo {WALLET_FILE} no encontrado")
    print("💡 Ejecuta la celda anterior para crear una nueva wallet")


### 1.3 Cargar Wallet desde Clave Privada

In [ ]:
# Cargar wallet desde clave privada
PRIVATE_KEY = "PRIVATE_KEY"

keypair = wallet_manager.load_wallet_from_private_key(PRIVATE_KEY)
if keypair:
    print(f"✅ Wallet cargada desde clave privada")
    print(f"📍 Dirección: {wallet_manager.get_address()}")

print("💡 Descomenta el código de arriba y agrega tu clave privada para usar esta función")


## 2. Información de Cuentas

Consultar balances, tokens y estado de cuentas.

### 2.1 Balance Completo de Wallet

In [ ]:
if wallet_manager.is_wallet_loaded():
    address = wallet_manager.get_address()
    balance_info = account_info.get_balance_info(address)
    
    print(f"\n💰 BALANCE COMPLETO")
    print(f"📍 Wallet: {address}")
    print(f"💎 SOL: {balance_info['sol_balance']:.9f}")
    print(f"💵 Valor USD: ${balance_info['usd_value']:.2f}")
    print(f"📊 Precio SOL: ${balance_info['sol_price_usd']:.2f}")
else:
    print("❌ No hay wallet cargada. Ejecuta las celdas de la sección 1 primero.")


### 2.2 Tokens SPL en la Wallet

In [ ]:
if wallet_manager.is_wallet_loaded():
    address = wallet_manager.get_address()
    token_accounts = account_info.get_token_accounts(address)
    
    if token_accounts:
        print(f"\n🪙 TOKENS SPL ENCONTRADOS: {len(token_accounts)}")
        print("=" * 60)
        
        for i, token in enumerate(token_accounts, 1):
            print(f"\n{i}. Token:")
            print(f"   📍 Mint: {token['mint']}")
            print(f"   💰 Balance: {token['balance']:,.{min(token['decimals'], 6)}f}")
            print(f"   🔢 Decimales: {token['decimals']}")
            print(f"   🏦 Cuenta: {token['account_address']}")
    else:
        print("📭 No se encontraron tokens SPL en esta wallet")
else:
    print("❌ No hay wallet cargada")


### 2.3 Información Detallada de Cuenta

In [ ]:
if wallet_manager.is_wallet_loaded():
    address = wallet_manager.get_address()
    account_details = account_info.get_account_info(address)
    
    print(f"\n🔍 INFORMACIÓN DETALLADA DE CUENTA")
    print("=" * 50)
    print(f"📍 Dirección: {account_details['address']}")
    print(f"💰 Balance SOL: {account_details['sol_balance']:.9f}")
    print(f"🏦 Lamports: {account_details['lamports']:,}")
    print(f"✅ Existe: {account_details['exists']}")
    print(f"🌐 Red: {account_details['network']}")
    
    if account_details.get('executable') is not None:
        print(f"⚙️ Ejecutable: {account_details['executable']}")
        print(f"👤 Owner: {account_details['owner']}")
        print(f"📊 Tamaño datos: {account_details['data_length']} bytes")
else:
    print("❌ No hay wallet cargada")


## 3. Transferencias SOL

Enviar SOL a otras wallets.

### 3.1 Estimar Comisiones de Transferencia

In [ ]:
# Estimar comisiones para una transferencia
AMOUNT_TO_SEND = 0.093827  # SOL

fee_estimate = transfer_manager.get_transfer_fee_estimate(AMOUNT_TO_SEND)

print(f"\n💸 ESTIMACIÓN DE COMISIONES")
print("=" * 40)
print(f"💰 Cantidad a enviar: {fee_estimate['amount_sol']} SOL")
print(f"💳 Comisión estimada: {fee_estimate['fee_sol']:.9f} SOL")
print(f"📊 Costo total: {fee_estimate['total_cost_sol']:.9f} SOL")
print(f"🌐 Red: {fee_estimate['network']}")


### 3.2 Transferir SOL

In [ ]:
# ⚠️ CUIDADO: Esta celda transfiere SOL real
# Configura los valores para usar

DESTINATION_ADDRESS = "GMN2f6PsBwUKKpuxQdsJGveJKsNEgiH7APVwdfBtnAtz"
AMOUNT_SOL = 0.093821719  # Cantidad en SOL

if wallet_manager.is_wallet_loaded():
    # Validar dirección destino
    if utils.validate_address(DESTINATION_ADDRESS):
        print(f"🔄 Iniciando transferencia de {AMOUNT_SOL} SOL...")
        
        signature = transfer_manager.transfer_sol(
            from_keypair=wallet_manager.keypair,
            to_address=DESTINATION_ADDRESS,
            amount_sol=AMOUNT_SOL
        )
        
        if signature:
            print(f"✅ Transferencia exitosa!")
            print(f"🔗 Signature: {signature}")
        else:
            print("❌ Error en la transferencia")
    else:
        print("❌ Dirección destino inválida")
else:
    print("❌ No hay wallet cargada")


## 4. Utilidades de Solana

Herramientas útiles para validación, conversiones y información de red.

### 4.1 Validar Direcciones

In [ ]:
# Validar direcciones de Solana
test_addresses = [
    "11111111111111111111111111111112",  # System Program
    "So11111111111111111111111111111111111111112",  # WSOL
    "direccion_invalida",  # Inválida
    "TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA"  # Token Program
]

print("🔍 VALIDANDO DIRECCIONES")
print("=" * 50)

for i, address in enumerate(test_addresses, 1):
    print(f"\n{i}. {address[:30]}...")
    is_valid = utils.validate_address(address)
    print(f"   Resultado: {'✅ Válida' if is_valid else '❌ Inválida'}")


### 4.2 Información de Red

In [ ]:
# Obtener información de la red Solana
network_info = utils.get_network_info()

if network_info:
    print(f"\n🌐 INFORMACIÓN DE RED")
    print("=" * 40)
    print(f"🌍 Red: {network_info['network']}")
    print(f"🔗 RPC URL: {network_info['rpc_url']}")
    print(f"📊 Slot actual: {network_info['current_slot']:,}")
    print(f"✅ Estado: {network_info['status']}")
    print(f"⏰ Timestamp: {network_info['timestamp']}")
else:
    print("❌ No se pudo obtener información de red")


### 4.3 Conversiones SOL ↔ Lamports

In [ ]:
# Conversiones entre SOL y lamports
test_values = {
    "sol_amounts": [0.001, 0.1, 1.0, 10.5],
    "lamport_amounts": [1000000, 100000000, 1000000000, 5000000000]
}

print("🔄 CONVERSIONES SOL ↔ LAMPORTS")
print("=" * 50)

print("\n📈 SOL → Lamports:")
for sol in test_values["sol_amounts"]:
    lamports = utils.convert_sol_to_lamports(sol)
    print(f"   {sol} SOL = {lamports:,} lamports")

print("\n📉 Lamports → SOL:")
for lamports in test_values["lamport_amounts"]:
    sol = utils.convert_lamports_to_sol(lamports)
    formatted = utils.format_balance(lamports)
    print(f"   {lamports:,} lamports = {formatted}")


### 4.4 Precio de SOL y Cálculos USD

In [ ]:
# Obtener precio actual de SOL y calcular valores
sol_amounts = [0.1, 1.0, 5.0, 10.0]

print("💵 PRECIO DE SOL Y VALORES USD")
print("=" * 50)

# Obtener precio actual
sol_price = utils.get_solana_price_usd()

if sol_price > 0:
    print(f"\n💰 Precio actual: ${sol_price:.2f} USD")
    print("\n📊 Valores calculados:")
    
    for amount in sol_amounts:
        value_usd = amount * sol_price
        print(f"   {amount} SOL = ${value_usd:.2f} USD")
else:
    print("❌ No se pudo obtener el precio de SOL")
    print("💡 Verifica tu conexión a internet")
